The current setup allows to execute PySpark operations on CERN Hadoop and Spark clusters. This notebook illustrates the use of Spark in SWAN to analyze the monitoring data available on HDFS and plots a heatmap of loadAvg across machines in a particular service.

### Connect to the cluster

To connect to a cluster, click on the star button on the top and follow the instructions
* The star button only appears if you have selected a SPARK cluster in the configuration
* The star button is active after the notebook kernel is ready

### Import necessary spark and python stuff

In [1]:
from pyspark.sql.functions import from_unixtime, when, col
from pyspark.sql.types import *
from pyspark.sql.functions import from_json

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data
*path_on_hdfs_to_your_data*

In [3]:
df = spark.read.json("/user/rucio01/traces/traces.2018*")

### check the structure

In [4]:
df.printSchema()

root
 |-- _corrupt_record: string (nullable = true)
 |-- account: string (nullable = true)
 |-- appid: string (nullable = true)
 |-- catStart: double (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- datasetName: string (nullable = true)
 |-- datasetScope: string (nullable = true)
 |-- dataset_scope: string (nullable = true)
 |-- duid: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filesize: string (nullable = true)
 |-- guid: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- ip: string (nullable = true)
 |-- localSite: string (nullable = true)
 |-- name: string (nullable = true)
 |-- pq: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- relativeStart: double (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- scope: string (nullable = true)
 |-- stateReason: string (nullab

### Filter for loadAvg
_https://metricmgr.cern.ch/metric/20002/_

In [5]:
df.head()

Row(_corrupt_record=None, account=None, appid=u'3772225596', catStart=1514761197.84869, clientState=u'DONE', dataset=u'panda.um.data17_13TeV.00340030.physics_Main.deriv.DAOD_BPHY6.f889_m1902_p3267_tid12608781_00.169436420', datasetName=None, datasetScope=None, dataset_scope=None, duid=None, eventType=u'get_sm', eventVersion=u'pilot3', filename=u'DAOD_BPHY6.12608781._000245.pool.root.1.panda.um_002', filesize=u'57392255', guid=u'4E82F48B91AAC946B8E897E3CCFE62C7', hostname=u'c-8-23.aglt2.org', ip=u'192.41.230.149', localSite=u'AGLT2_DATADISK', name=None, pq=u'AGLT2_SL6', protocol=u'lsm', relativeStart=1514761197.850914, remoteSite=u'AGLT2_DATADISK', scope=u'panda', stateReason=u'OK', suspicious=u'0', taskid=u'12608781', timeEnd=1514761199.339337, timeStart=1514761014.277093, traceId=u'7d679dc663e44880a812b0d709e9b7ee', traceIp=u'::ffff:192.41.230.149', traceTimeentry=u'2017-12-31T23:00:00.007341', traceTimeentryUnix=1514761200.007341, transferEnd=None, transferStart=1514761197.850914, ur

### Create temporary table view

In [6]:
# body_schema = spark.read.json(df_loadAvg.rdd.map(lambda row: row.body)).schema
df.createOrReplaceTempView("traces")

### Do the heavylifting in spark and collect aggregated view to panda DF

In [7]:
df_pandas = spark.sql("SELECT dataset, \
                               from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') AS dt, \
                               count(*) as amount \
                               FROM traces \
                               WHERE eventType = 'download' AND from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') >= date_add(current_date(), -30)\
                               GROUP BY 1, 2 \
                               HAVING count(1) BETWEEN 50 AND 10000\
                               ORDER BY 3 DESC")\
                    .toPandas()

In [8]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
df_pandas

dataset  \
0      mc15_13TeV.364370.Sherpa_222_NNPDF30NNLO_llvvjj_ss_EW6_CSSKIN.evgen.EVNT.e6582                                                                  
1      user.ljiang.data.data16_13TeV.00302053.physics_Main.DAOD.r9264_p3083_EXT0                                                                       
2      group.phys-higgs.mc16_13TeV.410470.CAOD_HIGG5D1.e6337_e5984_s3126_r10201_r10210_p3371.31-10_CxAOD.root                                          
3      user.adbailey.jetjet_JZ1.grid.AOD.LCJets.Run1_EXT0                                                                                              
4      group.phys-higgs.mc16_13TeV.363360.CAOD_HIGG5D1.e5983_e5984_s3126_r10201_r10210_p3371.31-10_CxAOD.root                                          
5      user.cylin.Pythia_JZ5W.MyTrk.v10_OUTPUT                                                                                                         
6      group.phys-higgs.mc16_13TeV.361028.CAOD_HIGG5D1.e3569_s3126_r10201_r10210_p3371.31-10_CxAOD.root                                                
7      data16_13TeV.periodD.physics_Main.PhysCont.DAOD_STDM3.grp16_v11_p3298                                                                           
8      group.phys-higgs.mc16_13TeV.361031.CAOD_HIGG5D1.e3569_s3126_r10201_r10210_p3371.31-10_CxAOD.root                                                
9      data16_13TeV.periodL.physics_Main.PhysCont.DAOD_STDM3.grp16_v11_p3298                                                                           
10     mc15_13TeV.364371.Sherpa_222_NNPDF30NNLO_llvvjj_ss_EW6_CKKW30.evgen.EVNT.e6582                                                                  
11     user.tnitta.mc15_14TeV.363655.MGPy8EG_N30NLO_Wtaunu_Ht140_280_CFilterBVeto.e9997.p3401.v1_DAOD_TRUTH1.363655.root                               
12     user.tnitta.mc15_14TeV.363660.MGPy8EG_N30NLO_Wtaunu_Ht500_700_CVetoBVeto.e9997.p3401.v1p1_DAOD_TRUTH1.363660.root                               
13     user.mverissi.NMF.fro_50_all_bins_td                                                                                                            
14     None                                                                                                                                            
15     user.tnitta.mc15_14TeV.363657.MGPy8EG_N30NLO_Wtaunu_Ht280_500_CVetoBVeto.e9997.p3401.v1_DAOD_TRUTH1.363657.root                                 
16     group.phys-higgs.mc16_13TeV.364170.CAOD_HIGG5D1.e5340_s3126_r10201_r10210_p3371.31-10_CxAOD.root                                                
17     user.tnitta.mc15_14TeV.363671.MGPy8EG_N30NLO_Wtaunu_Ht2000_E_CMS_BFilter.e9997.p3401.v1_DAOD_TRUTH1.363671.root                                 
18     user.tnitta.mc15_14TeV.363658.MGPy8EG_N30NLO_Wtaunu_Ht280_500_CFilterBVeto.e9997.p3401.v1_DAOD_TRUTH1.363658.root                               
19     user.tnitta.mc15_14TeV.363668.MGPy8EG_N30NLO_Wtaunu_Ht1000_2000_BFilter.e9997.p3401.v1_DAOD_TRUTH1.363668.root                                  
20     user.tnitta.mc15_14TeV.363661.MGPy8EG_N30NLO_Wtaunu_Ht500_700_CFilterBVeto.e9997.p3401.v1_DAOD_TRUTH1.363661.root                               
21     user.aleopold.hgtd_tt_mu200_vecRPT.2018_04_08_0010_HGTDHitAnalysis                                                                              
22     None                                                                                                                                            
23     user.tnitta.mc15_14TeV.363662.MGPy8EG_N30NLO_Wtaunu_Ht500_700_BFilter.e9997.p3401.v1p1_DAOD_TRUTH1.363662.root                                  
24     user.aezhilov.mc15_13TeV.361106.PowhegPythia8EvtGen_AZNLOCTEQ6L1_Zee.v11.1_histo.root                                                           
25     user.cylin.Pythia_JZ3W.MyTrk.v10_OUTPUT                                                                                                         
26     group.phys-higgs.mc16_13TeV.36

In [ ]:
df_pandas = spark.sql("SELECT dataset, from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') AS dt, \
                               count(*) \
                               FROM traces \
                               WHERE dataset != 'None' AND dataset != '' AND dataset != '[null]' AND eventType = 'download' AND from_unixtime(traceTimeentryUnix, 'YYYY-MM-dd') >= date_add(current_date(), -30)\
                               GROUP BY 1,2 \
                               ORDER BY 1")\
                    .toPandas()

In [ ]:


df_pandas.dtypes



In [ ]:
df_pandas['dt'] = df_pandas['dt'].astype('datetime64[ns]')

In [ ]:
df_pandas.dtypes

https://stackoverflow.com/questions/41494942/pandas-dataframe-groupby-plot

In [ ]:
df_pandas_sample = df_pandas['dataset'].sample(n=30, random_state=3)
ser = pd.DataFrame(df_pandas_sample.values)
ser.drop_duplicates()
ser.columns = ['dataset']
ser



In [ ]:
merged = df_pandas.merge(ser, on='dataset', how='inner', indicator=True)
temp = merged.copy(deep=True)
merged


In [ ]:
plt.plot(merged['dt'], merged['count(1)'], '*')
plt.xticks(rotation='vertical')

In [ ]:
merged.set_index('dt', inplace=True)
grouped = merged.groupby('dataset')

In [ ]:
gr = grouped.groups

In [ ]:
grouped['count(1)'].plot(legend=False, rot=90, figsize=(15,15))

In [ ]:
grouped['count(1)'].describe()